# Information Retrieval on Verses of the Holy Quran
Team Labs 2022 : 
Mohammed Abdul Khaliq,
Ufkun Bayram Menderes,
Muhammad Saad Magdi

## Imports

In [21]:
import warnings
warnings.filterwarnings('ignore')
import math

import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.stem import PorterStemmer

import collections
from collections import Counter
import more_itertools

import time

import nltk 
import regex as re



#IMPORTANT VARIABLES TO TINKER
use_loaded_inverted_index = 1 #change this to 0 to run the invertedindex code and not make use of preconstructed inverted index
use_loaded_tfidf = 1 #change this to 0 if you would like the program to compute tfidf from scratch (can take 2-3mins)
use_loaded_spacy_maps=1 #change this to 0 if you would like to run spacy mapping from scratch (can take >30mins)
use_verse_suggestion = 3 #(1 = verse similarity, 2 = preprocessed explanation similarity, 3 = unpreprocessed similarity)


subset_size = 1137 #the amount of rows to be taken from the humongous dataset (0 - for full dataset)
samples_invindex = 7 #the amount of samples of the inverted index to display (0 - for full inverted index)
samples_postlist = 7 #the amount of samples of postings list to display (0 - for full posting list)
samples_intersect = 7 #the amount of text samples of intersection to display (0 - all intersected tweets)




#nltk.download('punkt')
#nltk.download('stopwords')
english_sw = stopwords.words('english')



class PostList(list):

    def add_new_arrival(self, sublist):
        self.append(sublist)    #append tuple to self

    def message_count(self):
        return len(self)
    
#global postinglist
#postlinglist=PostList()

#postinglist = [[] for i in range(len(tokenwords))] #initialising an empty list of list which will be our posting list




In [22]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en 

!python3 -m spacy download en_core_web_lg

import spacy.cli
spacy.cli.download("en_core_web_lg")

You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/bin/python -m pip install --upgrade pip' command.
  Using cached en_core_web_sm-2.3.1-py3-none-any.whl
You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/en_core_web_sm
-->
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Using cached en_core_web_lg-2.3.1-py3-none-any.whl


You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
  Using cached en_core_web_lg-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/bin/python -m pip install --upgrade pip' command.


## Construction of Posting List and Inverted Index

In [2]:
#INDEX FUNCTION
def index(filename):
    
    print("Reading file...")
    
    global df
    global new_df
    
    df = pd.read_csv(filename)
    
    #print(df)
    
    if (subset_size == 0):
        new_df = df
    else:
        new_df = df[0:subset_size]
    print(new_df)
    
    #Preprocessing and adding new column "Clean Tweets" which will form our searchable tokens and "Original Text", which will preserve our text in string form.
    print("Removing NEWLINE and TAB")
    new_df['Text'] = new_df['Tafsir'].str.replace(r'\d+','')
    new_df["Text"] = new_df['Tafsir'].str.replace('NEWLINE','')
    new_df["Text"] = new_df['Tafsir'].str.replace('TAB','')
    new_df["Text"] = new_df['Tafsir'].str.replace('_','')
    
    
    print("Removing Punctuations")
    new_df["Text"] = new_df['Text'].str.replace('[^\w\s]','')
    new_df["Text"] = new_df['Text'].str.replace('\n','')
    new_df['Text']= new_df['Text'].str.lower()
    
    print("Removing stopwords")
    new_df['Clean_Text'] = new_df['Text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (english_sw)]))
    
    print("Tokenizing and creating new columns...")
    new_df['Original_Text']=new_df['Text']
    new_df['Text']=new_df['Text'].apply(word_tokenize)
    new_df['Clean_Text']=new_df['Clean_Text'].apply(word_tokenize)
    
    
    #Initialising global precedence to some important variables
    global counts
    global tokendict
    global tokendict1
    global tokenwords
    global postsizes
    
    tokenslist = new_df['Clean_Text'].to_list() #creating a list of list out of tweet tokens
    flatlist = list(np.concatenate(tokenslist)) #creating a 1D list to count all unique tokens
    counts = Counter(flatlist) #counting tokens
    
    tokendict = dict(counts) #creating a dictionary of key=token and value=occurences
    
    tokenwords = sorted(list(tokendict.keys())) #extracting only tokens from the dict and making a list of all token words in alphabetical order. This will also act as a hashmap to arrange a token to a specific id (index in this case).
    
    inv_dict={} #our soon to be inverted index

    postinglist=[[] for i in range(len(tokenwords))] #initialising an empty list of list which will be our posting list

    
    
    #Creating Posting List :
    '''
    The idea is to iterate over each row in the dataframe. We pick the tokens from the Clean_Text column
    and check the tokens of the original tweet to see if a token does exist.
    If it does exist, we make note of the index of the dataframe (document number) and see if the posting list already has a value
    for this document number at the position determined by tokenwords. The position determined by tokenwords will determine which of the lists
    of list corresponds to which token and will fill exactly that list whenever a token appears.
    '''
    
    for index, rows in new_df.iterrows():
    
        print('Creating Posting List : %d/100' %(round(index/1000)), end='\r')
    
        for item in new_df['Clean_Text'][index]:
        
            if item in new_df['Text'][index]:
                
                if index not in postinglist[tokenwords.index(item)]:
                    postinglist[tokenwords.index(item)].append(index)
                    
    #Posting List Created!
                    
    postsizes=[len(x) for x in postinglist] #Retrieves size of each of the lists for each token is posting list
    
    zip_iterator = zip(tokenwords, postsizes)


    tokendict1 = dict(zip_iterator) #Creates a dictionary with key = 'token' and value = 'size of posting list for that token'
        
    #NEW LINES
    postlist1 = PostList()
    postlist1.add_new_arrival(postinglist)
    
    #print(postlist1[0][2])
    print("Successfully created posting list!")
    
    
    #Creating Inverted Index
    '''
    The inverted index is now simply created as a dictionary with the following attributes:
    key = 'token' extracted from tokendict1(our zipped dict)
    value = a list of lists with the firstlist having one element equal to the 'size' of the posting list taken from postsizes and
            the second list having all the postings taken from postinglist.
    '''
    
    i=0
    for word in tokenwords:
        if word in tokendict:
            print('Building Inverted Index : %d/100' %(round((i/220540)*100)), end='\r')
            inv_dict[word]=[]
            # append the new number to the existing array at this slot
            inv_dict[word].append(tokendict1[word])
            #NEW LINE
            inv_dict[word].append(postlist1[0][tokenwords.index(word)])
        
            i=i+1
    
    print("Successfully Built Inverted Index")
                  
    return (inv_dict, postinglist)






## Query Function to Return Results From Inverted Index

In [3]:
#QUERY FUNCTION which takes two terms, by default term2=None in case only one argument is passed
def newest_query(termlist, explanation):
    
    
    #If only one argument is passed.
    if len(termlist)==1:
        chapverlist = []
        doclist=[]
        term1 = termlist[0]
        #print("\nPostings List for '" + term1 + "' is as follows :")
        #print(postlist[tokenwords.index(term1)])
        if term1 in tokenwords:
            for i in postlist[tokenwords.index(term1)]:
                if explanation==1:
                    print("\nChapter : " + str(new_df['Chapter'].iloc[i]))
                    print("VerseNo. : " + str(new_df['Verse'].iloc[i]))
                    print("\nVerse : " + str(new_df['Translation'].iloc[i]))
                    print("\nTafsir :" + str(new_df['Tafsir'].iloc[i]))
                    print("--------------------------------------------------")
                    
                chapver = '0'+ str(new_df['Chapter'].iloc[i]) + str(new_df['Verse'].iloc[i])
                chapverlist.append(chapver)
                doclist.append(i)
                
            
            return (chapverlist,doclist) #returns the postlist result for the specific token as determined by tokenwords (our basic hashmap) index for that word.
        else:
            return (chapverlist, doclist)
    
    
        
    else: 
        chapverlist=[]
        
        collectedpostlists = []
        
        for i in range (0, len(termlist)):
            #if postlist[tokenwords.index(termlist[i])]:
            if termlist[i] in tokenwords:
                collectedpostlists.append(postlist[tokenwords.index(termlist[i])])
            else:
                continue
            
                        
                        
        
        #collectedpostlists.append(postlist[tokenwords.index(termlist[i])])
            
            
        #postlist_t1 = postlist[tokenwords.index(term1)] #Retreive postlist of term1
        #postlist_t2 = postlist[tokenwords.index(term2)] #Retrieve postlist of term2
    
        #Get lengths of postlists
        #len_t1 = len(postlist_t1) 
        #len_t2 = len(postlist_t2)
    
        #Create an iterator for postlists
        #iter_t1 = iter(postlist_t1)
        #iter_t2 = iter(postlist_t2)
        
        if collectedpostlists:
            intersectlist=list(set.intersection(*[set(x) for x in collectedpostlists]))  #Our soon to be intersection of both postlists.
        else:
            return chapverlist
        
        #print(intersectlist)
        
        #print(intersectlist)
        #Initialising iterators with first value of both postlists
        #i1 = next(iter_t1) 
        #i2 = next(iter_t2)
        
        
        #PostList Intersection Logic
        '''
        We first check if both the iterators represent the same term. If they do, they get added to the intersection list.
        If not, the iterator with the smaller value goes to its next posting. This continues till both lists are exhausted.
        '''
        '''
        for i in range(0, len_t1+len_t2):
            if (i1==i2):
                intersectlist.append(i1)
                i1=next(iter_t1,0)
                i2=next(iter_t2,0)
            elif (i1>i2):
                i2=next(iter_t2,0)
            elif(i2>i1):
                i1=next(iter_t1,0)
        '''
        
        #PostList Intersection Completed
                    
        #print("\nIntersection List for '" + term1 +"' and '" + term2 +"' is as follows :")
        #print(intersectlist)
        
        #print("\nExamples : ")
        
        samples = new_df[['Chapter', 'Verse', 'Translation','Tafsir']].iloc[intersectlist]
        
            
        if (len(intersectlist)!=0):
            
            for i in range(0,len(intersectlist)):
                if explanation==1:
                    print("\nChapter : " + str(samples['Chapter'].iloc[i]))
                    print("VerseNo. : " + str(samples['Verse'].iloc[i]))
                    print("\nVerse :\n " + str(samples['Translation'].iloc[i]))
                    print("\nTafsir :\n" + str(samples['Tafsir'].iloc[i]))
                    print("--------------------------------------------------")
                
                chapver = '0'+ str(samples['Chapter'].iloc[i]) + str(samples['Verse'].iloc[i])
                chapverlist.append(chapver)
            
                #if(i==samples_intersect):
                #    break
        
        return(chapverlist, intersectlist)






# Constructing/Loading Inverted Index

In [4]:
import pickle

global postlist 

result=[]

tic = time.perf_counter()

invertedindex = {}
postlist=[]

if use_loaded_inverted_index == 1:
    print('Loading inverted index from file...')
    with open("../pickle/inverted_index", "rb") as fp:   # Unpickling
       invertedindex = pickle.load(fp)
    
    print('Loading postings list from file...')
    with open("../pickle/postlist", "rb") as fp:   # Unpickling
        postlist = pickle.load(fp)
        
    print('Loading relevant tokenwords from file...')
    with open("../pickle/tokenwords", "rb") as fp:   # Unpickling
        tokenwords = pickle.load(fp)
    
    print('Loading preprocessed dataset...')
    new_df = pd.read_pickle('../pickle/new_df')
    
else:
    invertedindex, postlist = index('../data/Quran.csv') #postlist stores the posting list, invertedindex stores the complete inverted index.


#Printing first n elements of each (samples_invindex variable under IMPORTANT VARIABLES) and showing how index matching is synchronous (hashmap between tokenwords and posting list is a success).
if (samples_invindex==0):
    print("Full inverted index:")
    print(inverted_index)
else:
    print("\n First %d elements of inverted index" %samples_invindex)
    n_items = more_itertools.take(samples_invindex, invertedindex.items())
    print(n_items)
        
if (samples_postlist == 0):
    print("Full postings list:")
    print(postlist)
else:
    print("\nFirst %d elements of posting list" %samples_postlist)
    print(postlist[:samples_postlist])
    
    
    
#result = query( "nation") #Returns complete postlist for a single term
#print(result)

#result = query( "sin", "forgive") #Returns intersection of postlist of both terms.

toc = time.perf_counter()
print(f"Completed in {toc - tic:0.4f} seconds")

    




Loading inverted index from file...
Loading postings list from file...
Loading relevant tokenwords from file...
Loading preprocessed dataset...

 First 7 elements of inverted index
[('0d', [1, [63]]), ('0l', [3, [145, 146, 147]]), ('1', [22, [0, 7, 202, 241, 261, 293, 294, 295, 296, 385, 386, 387, 493, 503, 669, 670, 789, 790, 791, 954, 955, 956]]), ('10', [18, [16, 302, 303, 499, 500, 501, 502, 675, 676, 677, 678, 679, 795, 796, 797, 798, 799, 963]]), ('100', [16, [105, 106, 107, 108, 109, 392, 393, 589, 590, 591, 592, 768, 769, 770, 888, 1053]]), ('1007', [3, [1125, 1126, 1127]]), ('101', [14, [105, 106, 107, 108, 109, 392, 393, 593, 768, 769, 770, 889, 1054, 1055]])]

First 7 elements of posting list
[[63], [145, 146, 147], [0, 7, 202, 241, 261, 293, 294, 295, 296, 385, 386, 387, 493, 503, 669, 670, 789, 790, 791, 954, 955, 956], [16, 302, 303, 499, 500, 501, 502, 675, 676, 677, 678, 679, 795, 796, 797, 798, 799, 963], [105, 106, 107, 108, 109, 392, 393, 589, 590, 591, 592, 768, 769

## Spelling Correction and Linguistic Variation

In [6]:
import pandas as pd 
import itertools
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

nlp =spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)
tokenizer = nlp.tokenizer

df=pd.read_csv('../data/Quran.csv')
tokens = set(list((itertools.chain(*[[t.text for t in tokenizer(row.lower()) if t.is_alpha] for row in df[~df.Tafsir.isna()].Tafsir.to_list()]))))


def levenshtein(token1, token2):
    size_x = len(token1) + 1
    size_y = len(token2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if token1[x-1] == token2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1] + 1,
                    matrix[x, y-1] + 1
                )
    # print (matrix)
    return (matrix[size_x - 1, size_y - 1])

def spell_recommendation(query_token):
    scores  = {}
    for t in tokens:
        lev_dist = levenshtein(t, query_token)
        if lev_dist <= 1:
            scores[t] = lev_dist
            print(scores)
    return [i[0] for i in sorted(scores.items(), key=lambda kv: (kv[1], kv[0]))]

In [7]:
variation_dict = {('aadam'):'adam',
                    ('eve', 'hava'):'hawa',
                    ('enoch', 'idrees', 'edris', 'idrissa'): 'idris',
                    ('noah', 'nooh'): 'nuh',
                    ('hud', 'hood') : 'hud',
                    ('saleh', 'shaleh', 'sawleh') : 'salih',
                    ('abraham', 'ibraheem', 'ebrahem', 'ebrahim'): 'ibrahim',
                    ('lot'): 'lut',
                    ('ishmael', 'ismaeel', 'ismayl'): 'ismail',
                    ('isaac', 'ishaaq') : 'ishaq',
                    ('jacob', 'yaqoob', 'jakob', 'jakov', 'yakoob', 'yakub') : 'yaqub',
                    ('joseph', 'yousuf', 'yousef', 'yosef', 'josef', 'yoseph', 'yusof') : 'yusuf',
                    ('job', 'ayub', 'ayoob', 'ayyoob'): 'ayyub',
                    ('jethro', 'suhaib', 'shoaib', 'suhayb', 'shuaib'): 'shu`ayb',
                    ('moses', 'musha', 'moosa') : 'musa',
                    ('pharoah', 'firaun', 'pharoh'):'firawn',
                    ('david', 'davud', 'dawid'): 'dawud',
                    ('sulaiman') : 'solomon',
                    ('elisha'): 'ilyas',
                    ('jonah'): 'yunus',
                    ('zachariah', 'zakkariya', 'zacharia', 'zakariah', 'zakkariah'): 'zakariyya',
                    ('john'): 'yahya',
                    ('eesa', 'jesus'): 'isa', 
                    ('bakka', 'bakkah', 'mecca', 'makka') : 'makkah',
                    ('mohammed', 'muhammed', 'mohammad', 'mohd'): 'muhammad'}

## Ranking
Ranking algorithm that makes use of the sum of substring matching (highest weight) and cosine similarity between query and result to reorder the results in order of decreasing relevance

In [8]:
clean_texts = new_df['Clean_Text'].to_list()


def tfidf(docfreq, no_of_docs, no_of_words, method=None):
    
    tfidf = {}
    #total_tokens = len(tokensfreq)

    for i in range(len(clean_texts)):
        text = clean_texts[i]
        total_tokens = len(text)
        occurences = Counter(text)
    
        for token in text:
           
            if (method==None or method == 'natural'):
                tf = occurences[token]/total_tokens
            elif method == 'log':
                tf = 1 + np.log(occurences[token]/total_tokens)
            df = docfreq[token]
            idf = np.log(len(clean_texts)/(df+1))
            tfidf[token] = tf*idf
        

    tfidf = collections.OrderedDict(sorted(tfidf.items()))
    return tfidf

docfreq = {}
no_of_docs = len(clean_texts)

for t in invertedindex:
    docfreq[t] = invertedindex[t][0]
    
no_of_words = len(docfreq)

tfidf = tfidf(docfreq, no_of_docs, no_of_words)
tfidf

OrderedDict([('0d', 0.01251085071635225),
             ('0l', 0.014124635331576613),
             ('1', 0.02108461771795344),
             ('10', 0.05245781428953969),
             ('100', 0.01811609978316517),
             ('1007', 0.00862573150019946),
             ('101', 0.014190486205404347),
             ('10101', 0.005415147175207684),
             ('10108', 0.011139842786270969),
             ('101611', 0.013049530121060278),
             ('102', 0.013592710609358592),
             ('1022', 0.00639847580139371),
             ('1028', 0.007068495482240984),
             ('103', 0.03864373475578862),
             ('1030', 0.007068495482240984),
             ('1031', 0.015684134628082182),
             ('10312musaylimah', 0.00382080836877891),
             ('103738وما', 0.00382080836877891),
             ('1038', 0.00382080836877891),
             ('104', 0.025144982563328925),
             ('1041', 0.014124635331576613),
             ('104let', 0.00780711402560617),
             

In [9]:
import pickle

wordDict = sorted(docfreq.keys())

def computeTFIDFVector(tweet):
    tfidfVector = [0.0] * len(wordDict)
    
    for i, word in enumerate(wordDict):
        if word in tweet:
            tfidfVector[i] = tfidf[word]
    return tfidfVector

        

if use_loaded_tfidf == 0:
    tfidfVector = [computeTFIDFVector(tweet) for tweet in clean_tweets]
else:
    with open("../pickle/tfIDF", "rb") as fp:   # Unpickling
       tfidfVector = pickle.load(fp)
    

tfidfVector[16]

[0.0,
 0.0,
 0.0,
 0.05245781428953969,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


In [10]:
def dot_product(vector_x, vector_y):
    dot = 0.0
    for e_x, e_y in zip(vector_x, vector_y):
        dot += e_x * e_y
    return dot

def magnitude(vector):
    mag = 0.0
    for index in vector:
        mag += math.pow(index, 2)
    return math.sqrt(mag)


def cosine_score(text1, text2=None):
    index_t1 = new_df["Original_Text"].str.contains(text1).idxmax()
    
    if text2 == None:
        
        sim_docs=[]
        score=[]
        doc_cos_pair ={}
       
        for i in range(len(tfidfVector)):
            cosim = dot_product(tfidfVector[index_t1], tfidfVector[i])/ (magnitude(tfidfVector[index_t1]) * magnitude(tfidfVector[i]))
            #sim_docs.append(cosim)
            
            doc_cos_pair[i] = cosim
            
        for w in sorted(doc_cos_pair, key=doc_cos_pair.get, reverse=True):
            sim_docs.append(w)
            score.append(doc_cos_pair[w])
                
            
        return(sim_docs[1:], score[1:])
            
        
    else:    
        index_t2 = new_df["Original_Text"].str.contains(text2).idxmax()
        cosim = dot_product(tfidfVector[index_t1], tfidfVector[index_t2])/ (magnitude(tfidfVector[index_t1]) * magnitude(tfidfVector[index_t2]))
    
    return cosim

In [11]:
def ranking(docids, query_list):
    
    relevantquery = ' '.join(query_list)
    #print(relevantquery)
    ranktuple = []
    
    for i in docids:
        score=0
        if relevantquery in (new_df['Original_Text'].iloc[i]):
            score+=1
        
        score+=cosine_score(relevantquery, new_df['Original_Text'].iloc[i])
        ranktuple.append((i,score))
        
        
    rankedtuple = sorted(ranktuple, key=lambda x: x[1], reverse=True)
    rankedverses = [x[0] for x in rankedtuple]
    
    chapverses_list = []
    for docs in rankedverses:
        chap = new_df['Chapter'].iloc[docs]
        verse = new_df['Verse'].iloc[docs]
        chapverses_list.append((chap,verse))
        
    
    return rankedverses, chapverses_list
        
        

## Boolean Retrieval 

In [12]:
def boolean_retrieval(query, explanation=0):
    print("Query :", query)
    
    query=query.lower()
    tokens = re.findall(r'([A-Za-z]+)',query)
    #print(tokens)
    

    #print(tokens)

    accepted_pos = ['VBN','VBG','VB','RP','RBS','RBR','RB','NNS','NNP','NN','JJS','JJR','JJ']

    query_words = nltk.pos_tag(tokens)
    #print(query_words)

    query_list =[]

    for i in range(0,len(query_words)):
        if (query_words[i][1] in accepted_pos) and (query_words[i][0]!='story') and (query_words[i][0]!='be') and (query_words[i][0]!='i') :
            query_list.append(query_words[i][0])
    
    
    for i in range(len(query_list)):
        if query_list[i] not in invertedindex.keys():   
            for key, value in variation_dict.items():
                if query_list[i] in key:
                    query_list[i] = variation_dict[key]
            
            corrected_token = spell_recommendation(query_list[i])

            if corrected_token:
                query_list[i]=corrected_token[0]

    chapvers,doclist = newest_query(query_list, explanation)
    
    #print(query_list)
    
    ranks = ranking(doclist, query_list)
    
    return query_list, chapvers, doclist

## Verse Suggestion 
In the following section, we will be implementing the feature of Verse suggestions. These suggestions are not part of the original results to a query, but rather further suggestions that might interest the user. The suggestions are calculated based on their similarity to other verses/explanations that are NOT in the result set but in the overall dataset.

### Remove List Duplicates function
The following cell will introduce a function to remove duplicates from a list without changing the order of the list. This will be important later on, where this functionality will be used to remove duplicate suggestions whilst keeping the order in tact, meaning that should a duplicate be encountered, the list will skip to the next most similar suggestion

In [13]:
def remove_list_duplicates(lst):
    """
    takes any list with any sort of elements and maintains only unique element of the input list
    while maintaining original list order
    
    Parameters
    ----------
    lst: list
        any list with any number of elements and any sort of elements
    
    Returns
    -------
    cleaned_list: list
        The original list with the same order where duplicate elements have been removed
    """
    seen_set = set()
    cleaned_list = [elem for elem in lst if elem not in seen_set and not seen_set.add(elem)]
    return cleaned_list

### Verse suggestion
The following cells will output suggested verses for our queries based on similarity between either verses or explanations from the result set.

First off, in the following cell, we're going to load the necessary verses and explanations.

In [14]:
import pickle

use_loaded_exp_prep_maps=1 #change this to 0 if you would like to run spacy mapping from scratch (can take >30mins)

disjoint_explanations_prep=[]
disjoint_explanations_unprep=[]
disjoint_verses=[]

if use_loaded_exp_prep_maps == 1:
    
    print('Loading spacy nlp objects from pickle files, this can take a moment...')
    
    with open("../pickle/disjoint_expl_prep", "rb") as fp:   # Unpickling
       disjoint_explanations_prep = pickle.load(fp)
    
    with open("../pickle/disjoint_expl_unprep", "rb") as fp:   # Unpickling
       disjoint_explanations_unprep = pickle.load(fp)
    
    with open("../pickle/disjoint_verses", "rb") as fp:   # Unpickling
       disjoint_verses = pickle.load(fp)
    
    print('Completed!')
else :
    
    
    all_explanations_prep = new_df["Clean_Text"].tolist()
    all_explanations_prep = [' '.join(ele) for ele in all_explanations_prep]
    disjoints_prep = all_explanations_prep.copy()
    disjoint_explanations_prep = [nlp(expl) for expl in disjoints_prep]
    
    all_explanations_unprep = new_df["Tafsir"].tolist()    
    disjoints_unprep = all_explanations_unprep.copy()
    disjoint_explanations_unprep = [nlp(expl) for expl in disjoints_unprep]
    
    all_verses = new_df["Translation"].tolist()
    dj_verses = all_verses.copy()
    disjoint_verses = [nlp(verse) for verse in dj_verses]

Loading spacy nlp objects from pickle files, this can take a moment...
Completed!


Now we will introduce a function to map the explanations to spacy large's vector space. Likewise, we will introduce a function to do the same for the verses in our dataset.

In [15]:
def expl_spacy_map(docids, use_plain_explanations=False):
    """
    maps the explanations in our dataset to spacy's vector space in the spacy large model
    
    Parameters
    ----------
    docids: list
        a list containing the id's of the documents that were returned as results to a query
    use_plain_explanations: bool
        determines whether the preprocessed (False) or the unpreprocessed versions of the 
        explanations will be used.
        The default is False.
    
    Returns
    -------
    result_expl_list: list
        a list of the result verses embedded in spacy's vector space
    all_explanations: list
        a list of all explanations embedded in spacy's vector space
    
    """
    
    if use_plain_explanations is True:
        #print('Using plain explanations...')
        all_explanations = new_df["Tafsir"].tolist()
    else: 
        all_explanations = new_df["Clean_Text"].tolist()
        all_explanations = [' '.join(ele) for ele in all_explanations]
        
    
    
    result_expl_list=[]
    
    for i in docids:
        #print(i)
        result_expl_list.append(nlp(all_explanations[i]))

        
    return result_expl_list, all_explanations

#result_expl_list, all_explanations = spacy_map1(ranks)

def verse_spacy_map(ranks):
    """
    maps all verses in our dataset into spacy's vector space in the large model
    
    Parameters
    ----------
    ranks: list
        a list of the ranked docids as results to a query
    
    Returns
    -------
    result_verse_list: list
        a list of the input ranks list mapped to spacy's vector space
    
    all_verses: list
        a list of all verses mapped to spacy's vector space
    
    """
    
    all_verses = new_df["Translation"].tolist()
    
    result_verse_list = []
    
    for i in ranks:
        #print(i)
        result_verse_list.append(nlp(all_verses[i]))
    
    return result_verse_list, all_verses



## Explanation-Similarity for Suggestion 
The following cell will introduce a function for the explanation-based suggestion.

In [16]:
#all_explanations = new_df["Clean_Text"].tolist()
#all_explanations = [' '.join(ele) for ele in all_explanations]

def max_explanation_sim_suggestion(docids, disjoint_explanations, result_expl_list,
                                   max_suggestions=True, print_message=True, return_chap_verse=True):
    
    """
    takes result verses of queries as input and suggests the most similar EXPLANATION as output
    for the top 3 of the input verse list

    Parameters
    ----------
    docids : list
        list of ids for the result verses of queries, can have arbitrary length
    disjoint_explanations: list
        list of all verses (i.e. their explanations) of 
        the results for a query have been "filtered out" by replacing their explanation.
        with an empty string. Every other remainining verse has a vector representation via spaCy's 
        nlp pipeline
    result_expl_list: list
        list of all verses (i.e. their explanations) in the result set for a query. These too have been 
        put through spaCy's nlp pipeline and have a vector representation.
    max_suggestions: bool
        determines whether only the top 3 suggestion or the entire list of suggestions (as long as docids)
        will be returned.
        The default is False.
    print_message: bool
        determines whether a message for the user suggesting top 3 verses should be returned.
        The default is True.
    return_chap_verse: False
        determines whether instead of the list of docids the list of chapter-verse tuples for the 
        suggestions will be returned.
        The default is False.

    Returns
    -------
    extended_docids: list
        list of original docids which is extended by exactly three elements, namely the top 3 suggestions.
        Docids themselves are integers.
    
    """
    
    expl_sims = []
    #print(result_expl_list[0])
    
    for result_expl in result_expl_list: 
        
        expl_similarities = [result_expl.similarity(expl) for expl in disjoint_explanations]
        expl_similarities[:] = [x if x != 1 else 0 for x in expl_similarities]
        expl_sims.append(expl_similarities)
        
    #print(expl_sims)
    
        
    #get most index of most similar verse for each verse in query result list
    most_sim_expl_index_list = [expl_list.index(max(expl_list)) for expl_list in expl_sims]
    most_sim_chapter_list = [new_df["Chapter"].iloc[expl] for expl in most_sim_expl_index_list]
    most_sim_verse_list = [new_df["Verse"].iloc[expl] for expl in most_sim_expl_index_list]
    most_sim_tuple_list = list(zip(most_sim_chapter_list, most_sim_verse_list))
    
    #remove duplicates --> since order will be preserved, list will skip to next highest ranked elem
    most_sim_expl_index_list_cleaned = remove_list_duplicates(most_sim_expl_index_list)
    most_sim_tuple_list_cleaned = remove_list_duplicates(most_sim_tuple_list)
    
    #append docids of top 3 suggestions to docids
    extended_docids = docids.copy()
    for index in most_sim_expl_index_list_cleaned[:3]:
        extended_docids.append(index)
    
    #set to True if more than 3 results suggestions shall be returned
    if max_suggestions is False:
        final_chap_verse_tup_list = most_sim_tuple_list_cleaned
        for elem in most_sim_expl_index_list_cleaned:
            extended_docids.append(elem)
    else:
        final_chap_verse_tup_list = most_sim_tuple_list_cleaned[:3]
    
    if print_message is False:
        return extended_docids
    #else:
        #print("You might also be interested in the following verses: \n")
        #print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in final_chap_verse_tup_list))
    
    if return_chap_verse is True:
        return final_chap_verse_tup_list
    else:
        return extended_docids

#max_explanation_suggestion = max_explanation_sim_suggestion(ranks, disjoint_explanations, result_expl_list)    

In [17]:
def inverted_explanation_results(query, disjoint_explanations, use_plain_explanations=False):
    """
    returns the results from the inverted index for input queries and suggests 
    based on explanations similarity
    
    Parameters
    ----------
    query: str
        Any input query (in our case those formulated in our dataset)
    disjoint_explanations list
        a list of all explanations which are NOT in the result set
    
    Returns
    -------
    chapvers: 
    
    max_explanation_suggestion: list
        a list of the ids of the verse suggestions
    Additionally, the function prints the suggested chapters and verses
    """
    
    query_dict = {}
    i=0
        
    query_list, chapvers, docids = boolean_retrieval(query)
    #print(docids)
    ranks, chapvers = ranking(docids, query_list)
        
    result_expl_list, all_explanations = expl_spacy_map(ranks, use_plain_explanations=use_plain_explanations)
        
    max_explanation_suggestion = max_explanation_sim_suggestion(ranks, disjoint_explanations, result_expl_list)
        
        
    for suggestions in max_explanation_suggestion:
        ranks.append(suggestions)
        
    query_dict[query] = ranks
        
    print('\n')
        
    return chapvers, max_explanation_suggestion

## Verse-Similarity for Suggestion
The following cell will introduce a verse-similarity-based function for suggestions.

In [18]:
#Similarity Verses

#all_verses = new_df['Translation'].to_list()

def max_verse_sim_suggestion(docids, disjoint_verses, result_verse_list, 
                             max_suggestions=True, print_message=True, return_chap_verse=True):
    """
    takes result verses of queries as input and suggests the most similar verse as output
    for the top 3 of the input verse list

    Parameters
    ----------
    docids : list
        list of ids for the result verses of queries, can have arbitrary length
    all_verses: list
        list of all verses where each element is a string, namely the verse itself
    max_suggestions: bool
        determines whether only the top 3 suggestion or the entire list of suggestions (as long as docids)
        will be returned.
        The default is False.
    print_message: bool
        determines whether a message for the user suggesting top 3 verses should be returned.
        The default is True.

    Returns
    -------
    extended_docids: list
        list of original docids which is extended by exactly three elements, namely the top 3 suggestions.
        Docids themselves are integers.
    
    """
        
    verse_sims = [] #create list of verse similarities for each verse
    for result_verse in result_verse_list:
        similarities = [result_verse.similarity(verse) for verse in disjoint_verses]
        similarities[:] = [v if v != 1 else 0 for v in similarities]
        verse_sims.append(similarities)
        
    #get max elements of each verse list only (for suggestions) --> most similar verse for each verse
    most_sim_index_list = [verse_list.index(max(verse_list)) for verse_list in verse_sims]
    #get chapter numbers of each most simlar verse
    most_sim_chapter_list = [new_df["Chapter"].iloc[most_sim] for most_sim in most_sim_index_list]
    #get verse numbers of each most similar verse
    most_sim_verse_list = [new_df["Verse"].iloc[most_sim] for most_sim in most_sim_index_list]
    most_sim_tuple_list = list(zip(most_sim_chapter_list, most_sim_verse_list))
    
    #remove duplicates --> since order will be preserved, list will skip to next highest ranked elem
    most_sim_index_list_cleaned = remove_list_duplicates(most_sim_index_list)
    most_sim_tuple_list_cleaned = remove_list_duplicates(most_sim_tuple_list)
       
    #append docids of top 3 suggestions to docids
    extended_docids = docids.copy()
    for index in most_sim_index_list_cleaned[:3]:
        extended_docids.append(index)
    
    #set to True if more than 3 results suggestions shall be returned
    if max_suggestions is False:
        final_chap_verse_tup_list = most_sim_tuple_list_cleaned
        for elem in most_sim_index_list_cleaned:
            extended_docids.append(elem)
    else:
        final_chap_verse_tup_list = most_sim_tuple_list_cleaned[:3]
    
    #decide whether message will be printed or not
    if print_message is False:
        return extended_docids
    #else:
        #print("You might also be interested in the following verses: \n")
        #print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in final_chap_verse_tup_list))
    
    if return_chap_verse is True:
        return final_chap_verse_tup_list
    else:
        return extended_docids
    
    
#verse_suggestion = max_verse_sim_suggestion(ranks, disjoint_verses, result_verse_list)

In [19]:
def inverted_verse_results(query, disjoint_verses):
    """
    returns the results from the inverted index for input queries
    
    Parameters
    ----------
    query: str
        Any input query (in our case those formulated in our dataset)
    disjoint_verses: list
        a list of all verses which are NOT in the result set
    
    Returns
    -------
    chapvers: 
    
    max_verse_suggestion: list
        a list of the ids of the verse suggestions
    Additionally, the function prints the suggested chapters and verses
    
    """
    
    query_dict = {}
    i=0
        
    query_list, chapvers, docids = boolean_retrieval(query)
    #print(docids)
    ranks, chapvers = ranking(docids, query_list)
        
    result_verse_list, all_verses = verse_spacy_map(ranks)
        
    max_verse_suggestion = max_verse_sim_suggestion(ranks, disjoint_verses, result_verse_list)
        
    list=[]
    for suggestions in max_verse_suggestion:
        #print(suggestions)
        list.append(suggestions)
        
    #print(ranks)
    query_dict[query] = ranks
        
    print('\n')
        
    return chapvers, max_verse_suggestion

## Main Code and Query

In [20]:
use_verse_suggestion = 3 #(1 = verse similarity, 2 = preprocessed explanation similarity, 3 = unpreprocessed similarity)

query = 'moses and pharoah'

if use_verse_suggestion == 1:
    
    results, suggestion = inverted_verse_results(query, disjoint_verses)
    
    print('Results :')
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in results))       
    
    print("\nYou might also be interested in the following verses: \n")
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in suggestion))
    
    
if use_verse_suggestion == 2:
    
    results, suggestion = inverted_explanation_results(query, disjoint_verses)
    #print(results)
    
    print('Results :')
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in results))      
    
    print("\nYou might also be interested in the following verses: \n")
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in suggestion))     
    
    
if use_verse_suggestion == 3:
    results, suggestion = inverted_explanation_results(query, disjoint_verses, use_plain_explanations=True)
    #print(results)
    
    print('Results :')
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in results))      
    
    print("\nYou might also be interested in the following verses: \n")
    print("\n".join("Chapter {}, Verse {}".format(*tup) for tup in suggestion)) 
    


Query : moses and pharoah


Results :
Chapter 1, Verse 2
Chapter 2, Verse 52
Chapter 2, Verse 53
Chapter 2, Verse 51

You might also be interested in the following verses: 

Chapter 4, Verse 171
Chapter 5, Verse 44
